In [1]:
import sys
import os

# Define the path to the modules directory
modules_path = os.path.abspath("../modules")

# Add the path to the sys.path if it's not already there
if modules_path not in sys.path:
    sys.path.append(modules_path)

# Import modules from the 'modules' package
from mamba import MambaModule
from rnn import RNNModule
from transformer import TransformerModule

# Other imports
import torch
import matplotlib.pyplot as plt
import numpy as np
import glob
from const import *
import yaml
import shutil
from pathlib import Path

/mnt/BigHD_1/loucas/miniconda3/envs/diss/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def copy_and_modify_yaml(input_path, output_dir, work_dir):
    """
    Copies a YAML file, modifies the resnet_checkpoint path, and returns the new path.

    Args:
    - input_path (str): The path to the input YAML file.
    - output_dir (str): The directory where the new YAML file will be saved.
    - work_dir (str): The new directory path to replace the resnet_checkpoint path.

    Returns:
    - str: The path to the new YAML file.
    """
    # Ensure output directory exists
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Read the input YAML file
    with open(input_path, "r") as file:
        yaml_data = yaml.safe_load(file)

    # Modify the resnet_checkpoint path
    if "resnet_checkpoint" in yaml_data:
        old_checkpoint_path = yaml_data["resnet_checkpoint"]
        filename = Path(old_checkpoint_path).name
        new_checkpoint_path = Path(work_dir) / filename
        yaml_data["resnet_checkpoint"] = str(new_checkpoint_path)

    # Create the new file path
    input_filename = Path(input_path).name
    new_filename = f"temp_{input_filename}"
    new_file_path = Path(output_dir) / new_filename

    # Write the modified data to the new YAML file
    with open(new_file_path, "w") as file:
        yaml.safe_dump(yaml_data, file)

    return str(new_file_path)

In [8]:
env = 0
start_idx = 10
file_idx = 10
sensors = [6,7]

In [19]:
filename = glob.glob(os.path.join(WORK_DIR, "data-w-camera/*.pt"))[file_idx]
raw_data = torch.load(filename, map_location=torch.device("cpu"))

In [20]:
sensor_data = raw_data["sensor_data"][start_idx:, env]
camera_data = raw_data["camera_data"][start_idx:, env]

print(sensor_data.shape)
print(camera_data.shape)

torch.Size([290, 35])
torch.Size([290, 3, 256, 256, 4])


In [21]:
model = RNNModule.load_from_checkpoint(
    checkpoint_path=RNN_LARGE_CHECKPOINT,
    hparams_file=copy_and_modify_yaml(RNN_LARGE_PARAMS, WORK_DIR, WORK_DIR),
)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
